# (1) Установка всех необходимых пакетов

In [1]:
import pymysql.cursors
import os
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, date, time, timedelta

# (2) Настройка связи с mysql

In [ ]:
conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="")
cur = conn.cursor()
cur.execute ("""USE orders;""")

# (3) Определение функций

In [121]:
def GetTicker ():
    print ('Сейчас Вам будет предложено ввести данные для заполнения книги заявок')
    correct = 0
    while correct == 0:
        seccode = input ('Введите тикер: ')
        cur.execute("SELECT count(*) FROM security_type WHERE (seccode = '"+ seccode +"');")
        if cur.fetchall() != [(0,)]:
            correct = 1
        else:
            print ('Вы ввели некорректный тикер. Повторите ввод')
    return(seccode)

def GetTime():
    correct = 0
    while correct == 0:
        time = int(input ('Введите момент времени (момент времени должен быть записан в формате HHMMSSQQQ): '))
        if int(time) in range(100000000,235959999):
            correct = 1
        else:
            print ('Вы ввели некорректный момент времени. Повторите ввод')
    return(time)

def DefineTable(seccode):
    cur.execute("SELECT instrument_type FROM security_type WHERE (seccode = '%s');" %(ticker))
    if cur.fetchall() == [('ordinary share',)]:
        table = 'ordshares'
    else:
        cur.execute("SELECT instrument_type FROM security_type WHERE (seccode = '%s');" % (ticker))
        if cur.fetchall() == [('preffered share',)]:
            table = 'prefshares'
        else:
            table = 'bonds'
    return(table)

def ReverseToDateTime(t):
    format = '%d.%m.%Y%H%M%S%f'
    time_str = dt.datetime.strptime(t,format)
    return time_str

def iceberg(data_from_postgres):
    import pandas as pd
    data = data_from_postgres
    #place - датафрейм (ДФ), который содержит наблюдения только по выставленным заявкам (ACTION = 1)
    #в ДФ содержатся данные по номеру заявки и выставленному объему (видимому)
    #iceberg = 1, если обнаружено, что выставленная заявка - айсберг. 0 - иначе
    #TIME_IDENT - время вскрытия заявки типа "айсберг". время указано для заявки, вскрывшей айсберг (снятие заявки или сделка) 
    place = data[data['ACTION']=='1']
    place = place[['ORDERNO', 'VOLUME']]
    place['ICEBERG'] = 0
    place['TIME_IDENT'] = 0
    place.index = place['ORDERNO']
    
    #iceberg - ДФ с данными по снятым заявкам и по сделкам (ACTION = 0 или ACTION = 2)
    iceberg = data[data['ACTION']!='1']
    iceberg ['VOLUME_INI'] = 0
    iceberg = iceberg.reset_index()
    iceberg = iceberg[['ORDERNO', 'VOLUME', 'TIME','VOLUME_INI']]
    
    #Идея: 
    #ДФ place содержит только уникальные наблюдения по каждому ORDERNO (ORDERNO - уникальный ключ для ДФ place)
    #для каждой строки ДФ iceberg находит соответсвующее значение ORDERNO в ДФ place
    #если колонка VOLUME в ДФ place больше нуля, но меньше объема снятия/сделки, то код вычитает объем снятой заявки/сделки.
    #при этом в колонке TIME_IDENT выставляется время снятия/сделки (так как это действие вскрыло айсберг)
    #в колонке iceberg ставится значение 1, что идентифицирует айсберг

    #если колонка VOLUME в ДФ place больше нуля и объема снятия/сделки, код вычитает объем снятой заявки/сделки.
    #это действие необходимо, чтобы правильно идентифицировать айсберг, если он будет вскрыт в дальнейшем
    #других действий не совершается

    #в остальных случаях (если айсберг не будет вскрыт / уже был вскрыт) никаких действий с таблицей place не совершается
    #P.S. в остальных случаях на самом деле прибавляется ноль к значению VOLUME в ДФ place, но это лишь технический момент
    #прибавление нуля не повлияет на результаты
    
    for i in range(len(iceberg)):

        order_number = iceberg.loc[i, 'ORDERNO']

        if (place.loc[order_number, 'VOLUME'] >= 0) and (iceberg.loc[i, 'VOLUME'] > place.loc[order_number, 'VOLUME']):
            place.loc[order_number,'VOLUME_INI'] = place.loc[order_number, 'VOLUME']
            place.loc[order_number, 'VOLUME'] -= iceberg.loc[i, 'VOLUME']
            place.loc[order_number, 'TIME_IDENT'] = iceberg.loc[i, 'TIME']
            place.loc[order_number, 'ICEBERG'] = 1

        elif (place.loc[order_number, 'VOLUME'] >= 0) and (iceberg.loc[i, 'VOLUME'] < place.loc[order_number, 'VOLUME']):
            place.loc[order_number,'VOLUME_INI'] = place.loc[order_number, 'VOLUME']
            place.loc[order_number, 'VOLUME'] -= iceberg.loc[i, 'VOLUME']

        else:
            place.loc[order_number,'VOLUME_INI'] = place.loc[order_number, 'VOLUME']
            place.loc[order_number, 'VOLUME'] += 0
            
    #ice_detected - ДФ, содержащий все вскрытые заявки типа "айсберг" за выбранный период времени
    #Примечание: значение VOLUME отражает объем, на котором айсберг был вскрыт. 
    #В ходе дальнейших торгов VOLUME мог и дальше уходить в отрицательную зону
    ice_detected = place[place['ICEBERG']==1]
    ice_detected.index.names = ['Index']
    
    #количество вскрытых айсбергов за выбранный период времени
    ice_numb = ice_detected['ICEBERG'].sum()
    
    #добавление времени выставления заявок, содержащих айсберги
    place = data[data['ACTION']=='1']
    place_chd = place[['ORDERNO', 'BUYSELL','PRICE', 'TIME', 'DATE','VOLUME_INI']]
    output = pd.merge(ice_detected, place_chd, on = 'ORDERNO', how = 'left')
    
    return output

# (4) Запрос пользовательских данных

In [113]:
# Запрос пользовательских данных
ticker = GetTicker()
table = DefineTable(ticker)
print('Сейчас Вам будет предложено ввести окончание периода анализа')
time_upper = GetTime()
print ('Время начала по умолчанию совпадает с временем начала основной торговой сессии - 10:00:00,000')
time_lower = 100000000
print ('Сейчас Вам будет предложено ввести шаг цены для выбранной бумаги')
price_step = float(input('Введите шаг цены (разделитель - точка): '))
print('Сейчас Вам будет предложено ввести дату расчета')
d = str(input('Введите дату расчета: '))

Сейчас Вам будет предложено ввести данные для заполнения книги заявок
Введите тикер: ALRS
Сейчас Вам будет предложено ввести окончание периода анализа
Введите момент времени (момент времени должен быть записан в формате HHMMSSQQQ): 103000000
Время начала по умолчанию совпадает с временем начала основной торговой сессии - 10:00:00,000
Сейчас Вам будет предложено ввести шаг цены для выбранной бумаги
Введите шаг цены (разделитель - точка): 0.01
Сейчас Вам будет предложено ввести дату расчета
Введите дату расчета: 29.04.2020


# (5) Запрос данных из БД и преобразование их в датафрейм

In [114]:
if table == 'ordshares':
    cur.execute("SELECT * from ordshares WHERE (seccode = '%s') and (ordertime <= '%s');" % (ticker, time_upper))
elif table == 'prefshares':
    cur.execute("SELECT * from prefshares WHERE (seccode = '%s') and (ordertime <= '%s');" % (ticker, time_upper))
rows = cur.fetchall()
data_from_postgres = pd.DataFrame(rows, columns =['NO','SECCODE','BUYSELL','TIME','ORDERNO','ACTION','PRICE','VOLUME','TRADENO','TRADEPRICE']) 
data_from_postgres ['DATE'] = d
data_from_postgres

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,DATE
0,2,ALRS,S,100000000,2,1,67.75,300,NaN,NaN,29.04.2020
1,87,ALRS,S,100000000,87,1,66.36,100,NaN,NaN,29.04.2020
2,313,ALRS,S,100000000,313,1,66.28,2000,NaN,NaN,29.04.2020
3,406,ALRS,B,100000000,406,1,65.00,100,NaN,NaN,29.04.2020
4,486,ALRS,S,100000000,486,1,69.63,900,NaN,NaN,29.04.2020
...,...,...,...,...,...,...,...,...,...,...,...
15198,1711319,ALRS,B,102955340,832302,0,65.94,100,NaN,NaN,29.04.2020
15199,1711347,ALRS,B,102955380,835896,1,66.00,100,NaN,NaN,29.04.2020
15200,1712246,ALRS,B,102956901,569932,0,65.55,1800,NaN,NaN,29.04.2020
15201,1712468,ALRS,B,102957264,836454,1,65.91,9000,NaN,NaN,29.04.2020


# (6) Расчет метрик для айсбергов

In [ ]:
output = iceberg(data_from_postgres)
output

/Users/ekaterina/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# (7) Расчет времени между выставлением и обнаружением айсберга

In [102]:
#сцепляем строки с датой и временем в одну строку
output['together_time'] = output['DATE']+output['TIME'].apply(str)
output['together_time_identified'] = output['DATE']+output['TIME_IDENT'].apply(str)
output['TIME'] = [ReverseToDateTime(output.together_time[i]) for i in range (len(output.together_time))]
output['TIME_IDENT'] = [ReverseToDateTime(output.together_time_identified[i]) for i in range (len(output.together_time_identified))]
output['DELTA'] = output['TIME_IDENT'] - output['TIME']
del output['together_time']
del output['together_time_identified']
output

,ORDERNO,VOLUME,iceberg,TIME_IDENT,BUYSELL,PRICE,TIME,DATE,DELTA
0,86829,-400,1,2020-04-29 10:01:42.388,S,65.64,2020-04-29 10:01:41.619,29.04.2020,00:00:00.769000
1,107058,-10500,1,2020-04-29 10:02:37.033,S,65.69,2020-04-29 10:02:22.955,29.04.2020,00:00:14.078000
2,113992,-7500,1,2020-04-29 10:02:38.747,S,65.69,2020-04-29 10:02:37.043,29.04.2020,00:00:01.704000
3,120434,-7000,1,2020-04-29 10:08:20.867,S,65.69,2020-04-29 10:02:53.305,29.04.2020,00:05:27.562000
4,147575,-3400,1,2020-04-29 10:04:09.587,B,65.51,2020-04-29 10:03:59.198,29.04.2020,00:00:10.389000
...,...,...,...,...,...,...,...,...,...
163,6731494,-3400,1,2020-04-29 17:48:31.195,S,65.98,2020-04-29 17:48:19.266,29.04.2020,00:00:11.929000
164,6734093,-200,1,2020-04-29 17:48:31.249,B,65.93,2020-04-29 17:48:31.197,29.04.2020,00:00:00.052000
165,6805069,-1300,1,2020-04-29 17:55:49.478,B,66.05,2020-04-29 17:55:49.478,29.04.2020,00:00:00
166,6805439,-3000,1,2020-04-29 17:55:52.243,B,66.05,2020-04-29 17:55:52.235,29.04.2020,00:00:00.008000


In [4]:
def lob_almost(data): 
    orders = data
    # Построение книги лимитированных заявок (LOB - limited order book)
    # Устанавливаем маску лимитных заявок заявках (True - лимитная заявка)
    no_mrk_ind=orders['PRICE']!=0
    #  Таблица выставленных заявок
    plc=orders[['ORDERNO','BUYSELL','PRICE','VOLUME']][(orders['ACTION']=='1') & no_mrk_ind]
    # Таблица отозванных заявок
    wdr=orders[['ORDERNO','VOLUME']][(orders['ACTION']=='0') & no_mrk_ind]
    # Таблица исполненных объемов
    trds=orders[['ORDERNO','VOLUME']][(orders['ACTION']=='2') & no_mrk_ind] 
    trds=trds[['ORDERNO','VOLUME']].groupby(['ORDERNO']).sum()
    trds['ORDERNO']=trds.index
    trds.index=range(trds.shape[0])
    # Формируем книгу заявок через мерж всех рассчитанных показателей
    lob_almost=pd.merge(plc,wdr, on='ORDERNO',how='left',suffixes=('_plc','_wdr'))
    lob_almost=pd.merge(lob_almost,trds, on='ORDERNO', how='left')
    lob_almost=lob_almost.rename(columns={'VOLUME':'VOLUME_trds'})
    lob_almost=lob_almost.fillna(0)
    lob_almost['TOTAL_VOL']=lob_almost['VOLUME_plc']-lob_almost['VOLUME_wdr']-lob_almost['VOLUME_trds']
    lob_almost = lob_almost[['ORDERNO','BUYSELL','PRICE','VOLUME_plc','VOLUME_wdr','VOLUME_trds','TOTAL_VOL']][(lob_almost['TOTAL_VOL']!=0)]
    return lob_almost

def place(data_from_postgres):
    # Строим сводную таблицу с объемами на постановку, снятие и сделки
    # это почти стакан, но не законченный
    lob = lob_almost(data_from_postgres)
    
    # Вытаскиваем обыкновенные заявки
    lob_ordinar = lob[lob['TOTAL_VOL']>0].pivot_table(index = "PRICE", columns = "BUYSELL", values = "TOTAL_VOL", aggfunc = 'sum')
    lob_ordinar.sort_index(axis = 0, inplace = True, ascending = False)
    lob_ordinar = lob_ordinar.fillna(0)
    
    # Вытаскиваем заявки айсберги
    lob_ice = lob[lob['TOTAL_VOL']<0].pivot_table(index = "PRICE", columns = "BUYSELL", values = "TOTAL_VOL", aggfunc = 'sum')
    lob_ice.sort_index(axis = 0, inplace = True, ascending = False)
    lob_ice = lob_ice.fillna(0)
    
    # Сводим все в одной таблице
    lob_fin = pd.merge(lob_ordinar, lob_ice, on = 'PRICE', how = 'outer', suffixes=('','_ice'))
    lob_fin.sort_index(axis = 0, inplace = True, ascending = False)
    lob_fin = lob_fin.fillna(0)
    
    plt.barh(lob_fin.index, -lob_fin['B'], color = 'g', height = 0.2, label = 'Buy')
    plt.barh(lob_fin.index, lob_fin['S'], color = 'r', height = 0.2, label = 'Sell')
    plt.barh(lob_fin.index, -lob_fin['B_ice'], color = 'y', height = 0.2, label = 'Buy_Ice')
    plt.barh(lob_fin.index, lob_fin['S_ice'], color = 'b', height = 0.2, label = 'Sell_Ice')
    plt.legend()



def bid_ask(data_from_postgres, time):

    lob_ = lob_almost(data_from_postgres[data_from_postgres['TIME'] <= time])
    # Вытаскиваем обыкновенные заявки
    lob = lob_[lob_['TOTAL_VOL']>0].pivot_table(index = "PRICE", columns = "BUYSELL", values = "TOTAL_VOL", aggfunc = 'sum')
    lob.sort_index(axis = 0, inplace = True, ascending = False)
    lob = lob.fillna(0)
    
    bid = lob[lob['B'] > 0].index[0]
    ask = lob[lob['S'] > 0].index[-1]
    bid_ask = {'bid': bid, 'ask': ask}
    
    return bid_ask

def place_from_ask_bid(output, price_step):
    """
    Это метод, а не функция, поэтому не возвращает значения
    Значит, нужно просто вызвать метод, дав на вход предыдущий аутпут
    Присваивать значение оператором "=" не надо
    """
    
    output['Bid/Ask'] = 0
    for i in output.index:
        bid = output.loc[i, 'BUYSELL'] == 'B'
        price = output.loc[i, 'PRICE']
        bid_or_ask = bid_ask(data_from_postgres, output.loc[i, 'TIME'] * 1000)
        output.loc[i, 'Bid/Ask'] = bid_or_ask['bid'] - price if bid else price - bid_or_ask['ask']
    output['numb_of_steps'] = output['Bid/Ask'] / price_step
    
def time_to_detection(output):
    
    import matplotlib.pyplot as plt
    
    delta_0 = len(output[output['delta']==0])
    delta_less_10 = len(output[(output['delta']>0)&(output['delta']<=10)])
    delta_10_30 = len(output[(output['delta']>10)&(output['delta']<=30)])
    delta_30_60 = len(output[(output['delta']>30)&(output['delta']<=60)])
    delta_more_60 = len(output[output['delta']>60])
    plt.bar('0', delta_0, color = 'lime')
    plt.bar('0-10', delta_less_10, color = 'g')
    plt.bar('10-30', delta_10_30, color = 'yellow')
    plt.bar('30-60', delta_30_60, color = 'orange')
    plt.bar('>60', delta_more_60, color = 'red')
    plt.xlabel('Время до обнаружения айсберга после его выставления')
    plt.ylabel('Количество заявок')

# (5) Таблица с айсбергами и характеристиками заявки

In [15]:
output = iceberg(data_from_postgres)
output

,ORDERNO,VOLUME,iceberg,time_identified,BUYSELL,PRICE,TIME,delta
0,86829,-400,1,100142,S,65.64,100142,0
1,107058,-10500,1,100237,S,65.69,100223,14
2,113992,-7500,1,100239,S,65.69,100237,2
3,120434,-7000,1,100821,S,65.69,100253,328
4,147575,-3400,1,100410,B,65.51,100359,11
...,...,...,...,...,...,...,...,...
163,6731494,-3400,1,174831,S,65.98,174819,12
164,6734093,-200,1,174831,B,65.93,174831,0
165,6805069,-1300,1,175549,B,66.05,175549,0
166,6805439,-3000,1,175552,B,66.05,175552,0


# (6) Добавление расстояния от Bid/Ask в ценах на момент выставления заявки¶

In [16]:
place_from_ask_bid(output, price_step)
output

,ORDERNO,VOLUME,iceberg,time_identified,BUYSELL,PRICE,TIME,delta,Bid/Ask,numb_of_steps
0,86829,-400,1,100142,S,65.64,100142,0,0.05,5.0
1,107058,-10500,1,100237,S,65.69,100223,14,0.01,1.0
2,113992,-7500,1,100239,S,65.69,100237,2,0.03,3.0
3,120434,-7000,1,100821,S,65.69,100253,328,0.00,0.0
4,147575,-3400,1,100410,B,65.51,100359,11,0.07,7.0
...,...,...,...,...,...,...,...,...,...,...
163,6731494,-3400,1,174831,S,65.98,174819,12,0.00,0.0
164,6734093,-200,1,174831,B,65.93,174831,0,-0.01,-1.0
165,6805069,-1300,1,175549,B,66.05,175549,0,-0.04,-4.0
166,6805439,-3000,1,175552,B,66.05,175552,0,0.00,0.0


In [48]:
from datetime import datetime
d = '29.11.2020'
def ReverseToDatetime(date, ordertime):
    hh24 = ordertime//10000000 # выделяем разряды, соответствующие часам
    mi = ordertime//100000%100 # выделяем разряды, соответствующие минутам
    ss = ordertime//1000%100 # выделяем разряды, соответствующие секундам
    ff3 = ordertime%1000 # выделяем разряды, соответствующие микросекундам
    reversed_time = datetime.strptime(date + ' ' + str(hh24) + ':' + str(mi) + ':' + str(ss) + ',' + str(ff3), '%d.%m.%Y %H:%M:%S,%f') #переводим строку в формат времени для python
    return(reversed_time)